# Colab Training Test - Qwen3 0.8B on Linux Commands

**Purpose:** Compare Colab Pro vs Kaggle for training ease and speed

**Dataset:** AnishJoshi/nl2bash-custom (Linux commands)

**Expected Time:** 2-3 hours on A100

In [ ]:
# Install dependencies
!pip install -q transformers datasets accelerate peft bitsandbytes trl torch

print("\n✅ Installation complete")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 30.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.6/564.6 kB 45.1 MB/s eta 0:00:00

✅ Installation complete


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from trl import SFTTrainer, SFTConfig

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    print(f"GPU Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

PyTorch version: 2.8.0+cu126
CUDA available: True
GPU: NVIDIA A100-SXM4-80GB
GPU Memory: 85.17 GB


In [ ]:
# Configuration
MODEL_NAME = "DavidAU/Qwen3-Zero-Coder-Reasoning-0.8B"
OUTPUT_DIR = "./qwen3-colab-linux-test"
DATASET_NAME = "AnishJoshi/nl2bash-custom"

# Training hyperparameters
BATCH_SIZE = 4
GRADIENT_ACCUMULATION = 4
LEARNING_RATE = 2e-4
NUM_EPOCHS = 3
MAX_SEQ_LENGTH = 2048

# LoRA config
LORA_R = 16
LORA_ALPHA = 32
LORA_DROPOUT = 0.05

# CRITICAL: Same padding token
PAD_TOKEN_ID = 151645

print("✅ Configuration set")

✅ Configuration set


In [ ]:
# Load tokenizer
print("Loading tokenizer from HuggingFace...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, trust_remote_code=True)

# CRITICAL: Force same padding token
tokenizer.pad_token_id = PAD_TOKEN_ID
tokenizer.padding_side = "right"

print(f"✅ Tokenizer loaded")
print(f"Pad token ID: {tokenizer.pad_token_id}")

Loading tokenizer from HuggingFace...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

✅ Tokenizer loaded
Pad token ID: 151645


In [ ]:
# Load model with 4-bit quantization
print("="*60)
print("Loading Model from HuggingFace")
print("="*60)

print("\nConfiguring 4-bit quantization...")
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True
)

print(f"Loading {MODEL_NAME}...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

print("Preparing model for LoRA training...")
model = prepare_model_for_kbit_training(model, use_gradient_checkpointing=True)

print(f"✅ Model loaded on device: {model.device}")

Loading Model from HuggingFace

Configuring 4-bit quantization...
Loading DavidAU/Qwen3-Zero-Coder-Reasoning-0.8B...


config.json:   0%|          | 0.00/822 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Preparing model for LoRA training...
✅ Model loaded on device: cuda:0


In [ ]:
# Configure LoRA
print("\nApplying LoRA configuration...")
lora_config = LoraConfig(
    r=LORA_R,
    lora_alpha=LORA_ALPHA,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=LORA_DROPOUT,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
print("\n✅ LoRA applied")
model.print_trainable_parameters()


Applying LoRA configuration...

✅ LoRA applied
trainable params: 6,881,280 || all params: 823,164,416 || trainable%: 0.8360


In [ ]:
# Load Linux commands dataset
print("Loading Linux commands dataset...")
dataset = load_dataset(DATASET_NAME, split="train")
print(f"Dataset size: {len(dataset)} examples")
print(f"\nSample entry:")
print(dataset[0])

Loading Linux commands dataset...


README.md: 0.00B [00:00, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


train.json: 0.00B [00:00, ?B/s]

dev.json: 0.00B [00:00, ?B/s]

test.json: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/19658 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2457 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2458 [00:00<?, ? examples/s]

Dataset size: 19658 examples

Sample entry:
{'bash_code': 'find -mtime +2 -mtime -5', 'nl_command': 'Look for any files that were modified 2-5 days ago', 'srno': 3194}


In [ ]:
# Format dataset for Linux command training
def format_instruction(example):
    description = example.get('description', example.get('question', ''))
    command = example.get('cmd', example.get('answer', ''))

    prompt = f"Instruction: {description}\n\nResponse:"
    text = f"{prompt} {command}"
    return {"text": text}

dataset = dataset.map(format_instruction)
print("\nFormatted example:")
print(dataset[0]['text'][:500])

Map:   0%|          | 0/19658 [00:00<?, ? examples/s]


Formatted example:
Instruction: 

Response: 


In [ ]:
# Training arguments
training_args = SFTConfig(
    output_dir=OUTPUT_DIR,
    num_train_epochs=NUM_EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION,
    learning_rate=LEARNING_RATE,
    logging_steps=10,
    save_strategy="epoch",
    fp16=True,
    optim="paged_adamw_8bit",
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",
    report_to="none",
    dataset_text_field="text",
    packing=False,
)

print("✅ Training arguments configured")

✅ Training arguments configured


In [ ]:
# Create trainer and start training
print("\n" + "="*60)
print("STARTING TRAINING")
print("="*60)

trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

print(f"\nTraining on {len(dataset)} examples...")
print(f"Total steps: {len(dataset) // (BATCH_SIZE * GRADIENT_ACCUMULATION) * NUM_EPOCHS}")
print("\nStarting training...\n")

trainer.train()

print("\n" + "="*60)
print("✅ TRAINING COMPLETE!")
print("="*60)


STARTING TRAINING


Adding EOS to train dataset:   0%|          | 0/19658 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/19658 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/19658 [00:00<?, ? examples/s]


Training on 19658 examples...
Total steps: 3684

Starting training...



Step,Training Loss
10,6.626400
20,5.164800


In [ ]:
# Move LoRA parameters to GPU
model = model.to("cuda")
print(f"✅ Model moved to GPU: {next(model.parameters()).device}")

In [ ]:
# Save the trained LoRA adapter
print("\nSaving LoRA adapter...")
model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

import os
adapter_path = f"{OUTPUT_DIR}/adapter_model.safetensors"
if os.path.exists(adapter_path):
    adapter_size = os.path.getsize(adapter_path) / 1e6
    print(f"✅ Adapter saved: {adapter_size:.1f} MB")
else:
    print(f"❌ ERROR: Adapter file not found")

# Summary

**Colab vs Kaggle Comparison:**

Record your observations:
- Setup time
- Training time
- Ease of use
- GPU type assigned
- Any errors encountered
- Overall experience